# Running the Notebook Server

The Jupyter notebook server runs Notebooks on the web.

## Starting a notebook server using the command line

You can start the notebook server from the command line (Terminal on Mac/Linux, CMD prompt on Windows) by running the following command: 

    jupyter notebook

This will print some information about the notebook server in your terminal, including the URL of the web application (by default, `http://127.0.0.1:8888`). It will then open your default web browser to this URL.

When the notebook opens, you will see the **notebook dashboard**, which will show a list of the notebooks and subdirectories in the directory where the notebook server was started. As of IPython 2.0, the dashboard allows you to navigate to different subdirectories. Because of this, it is no longer necessary to start a separate notebook server for each subdirectory. Most of the time, you will want to start a notebook server in the highest directory in your filesystem where notebooks can be found. Often this will be your home directory.

You can start more than one notebook server at the same time. By default, the first notebook server starts on port 8888 and later notebook servers search for open ports near that one.

You can also specify the port manually:

    jupyter notebook --port 8883

Or start notebook server without opening a web browser.

    jupyter notebook --no-browser

The notebook server has a number of other command line arguments that can be displayed with the `--help` flag: 

    jupyter notebook --help

<div class="alert alert-failure">
 
</div>



## Configuring the IPython Notebook

The notebook web server can also be configured using IPython profiles and configuration files. The Notebook web server configuration options are set in a file named `ipython_notebook_config.py` in your IPython *profile directory*. The profile directory is a subfolder of your IPython directory, which itself is usually `.ipython` in your home directory.

You can display the location of your default profile directory by running the command:

In [7]:
!jupyter --paths

/Users/bgranger/.ipython/profile_default


To start the notebook server using a given profile, run the following:

    jupyter notebook --profile=my_profile

More details about Jupyter configuration files and profiles can be found [here](https://jupyter-notebook.readthedocs.io/en/stable/config.html).

## Running a public notebook server

<div class="alert alert-error">
Don't run a public notebook server unless you know what you are doing.
</div>

By default the notebook server only listens on the `localhost/127.0.0.1` network interface. If you want to connect to the notebook from another computers, or over the internet, you need to configure the notebook server to listen on all network interfaces and not open the browser. You will often also want to disable the automatic launching of the web browser.

This can be accomplished by passing a command line options.

    jupyter notebook --ip=* --no-browser

You can also add the following to your`ipython_notebook_config.py` file:

```python
c.NotebookApp.ip = '*'
c.NotebookApp.open_browser = False
```

## Running with a different URL prefix

The notebook dashboard typically lives at the URL `http://localhost:8888/tree`. If you prefer that it lives, together with the 
rest of the notebook web application, under a base URL prefix, such as `http://localhost:8888/ipython/tree`, you can do so by adding the following lines to your `ipython_notebook_config.py` file.

```python
c.NotebookApp.base_url = '/ipython/'
c.NotebookApp.webapp_settings = {'static_url_prefix':'/ipython/static/'}
```

## Known issues

When behind a proxy, especially if your system or browser is set to autodetect the proxy, the notebook web application might fail to connect to the server's websockets, and present you with a warning at startup. In this case, you need to configure your system not to use the proxy for the server's address.

For example, in Firefox, go to the Preferences panel, Advanced section,
Network tab, click 'Settings...', and add the address of the notebook server
to the 'No proxy for' field.

## Using a different notebook store

By default, the notebook server stores the notebook documents that it saves as files in the working directory of the notebook server, also known as the
`notebook_dir`. This  logic is implemented in the `FileNotebookManager` class. However, the server can be configured to use a different notebook manager class, which can store the notebooks in a different format. 

The [bookstore](https://github.com/rgbkrk/bookstore) package currently allows users to store notebooks on Rackspace CloudFiles or OpenStack Swift based object stores.

Writing a notebook manager is as simple as extending the base class `NotebookManager`. The [simple_notebook_manager](https://github.com/khinsen/simple_notebook_manager) provides a great example
of an in memory notebook manager, created solely for the purpose of
illustrating the notebook manager API.

# Multi-User Notebook Servers

## JupyterHub

JupyterHub is a multi-user server that manages and proxies multiple instances of the single-user IPython/Jupyter Notebook server. (see also Bindr project)

<img src="https://c2a32ff18d23c8f567f0-e44b0df73868b5d567b1e58e01681d15.ssl.cf5.rackcdn.com/2015-03-24-deploying-jupyterhub-for-education/jupyterhub-9efc59baf33d2640641cb4a1fd9145ff.gif" width="800px" />

### Components

* multi-user Hub (tornado process)
* configurable http proxy (node-http-proxy)
* multiple single-user IPython notebook servers (Python/IPython/tornado)

#### [Configurable Proxy](https://github.com/jupyter/configurable-http-proxy)

* Based on node-http-proxy
* Proxy routes to localhost, external IPs, etc.

```
/user/123/ -> 127.0.0.1:56790
```

* Provides an administrative API for adding routes

```
POST /api/routes/[:path]
```

#### [JupyterHub](https://github.com/jupyter/jupyterhub)

* Hub spawns proxy
* Proxy forwards ~all requests to hub by default
* Hub handles login, and spawns single-user servers on demand
* Hub configures proxy to forward url prefixes to single-user servers


## Authentication models

* PAM/Unix (Default)
* [GitHub OAuth](https://github.com/jupyter/oauthenticator) - Demo at demohub.jupyter.org

This is extensible enough to implement other authentication methods, the simplest being any involving OAuth (follow the GitHub OAuthenticator for a skeleton).

## Spawners

By default, notebook servers are spawned in the context of the user as a process on the host machine. One alternative for spawning is the [DockerSpawner](https://github.com/jupyter/dockerspawner) which runs each user in their own environment inside a Docker container.


## Real world implementations of JupyterHub

The Computational Models class at UC Berkeley ran a JupyterHub installation for ~220 students in Winter/Spring of 2015.

* Docker Spawner
* Multiple compute nodes
* GitHub Authentication
* NFS backed for student data, assignments, notebooks, etc.

![](https://c2a32ff18d23c8f567f0-e44b0df73868b5d567b1e58e01681d15.ssl.cf5.rackcdn.com/2015-03-24-deploying-jupyterhub-for-education/setup-2e74d935ee0c874e66a9b53359493ceb.png)


## tmpnb

tmpnb is a temporary notebook system. Visit [try.jupyter.org](https://try.jupyter.org) for a demo.

Similar to JupyterHub with the DockerSpawner, tmpnb uses a proxy to route to notebook servers. The difference is that there is no login and notebook servers get deleted after a period of (configurable) inactivity.

<img src="https://cloud.githubusercontent.com/assets/836375/5911140/c53e3978-a587-11e4-86a5-695469ef23a5.png" width="800px" />
